In [1]:
import ast
import random

import pandas as pd

df = pd.read_csv('../output_data/detailed_travel_routes.csv')

# Parse columns
df['Domain'] = df['Domain'].apply(lambda x: ast.literal_eval(x.replace('coordinates = ', '').strip()))
df['Plan'] = df['Plan'].apply(ast.literal_eval)
df['Detailed Plan'] = df['Detailed Plan'].apply(ast.literal_eval)

df

,id,Domain,Goal,Plan,Detailed Plan
0,0,"[[-71.22407723, 42.20650415], [-71.06341724278...",WESTWOOD/PEMBROKE HEALTH SYSTEM WESTWOOD (WEST...,"[[Drive north on Abbey Road., Turn left onto C...",[[{'instruction': 'Drive north on Abbey Road.'...
1,1,"[[-80.31080036, 26.01301541], [-80.21063654, 2...","MEMORIAL HOSPITAL WEST (PEMBROKE PINES, FL) to...","[[Drive northwest., Turn left after 19 meters ...","[[{'instruction': 'Drive northwest.', 'name': ..."
2,2,"[[-98.57619951, 29.52010675], [-97.7136084, 30...","CLARITY CHILD GUIDANCE CENTER (SAN ANTONIO, TX...","[[Drive east., Turn left onto Tom Slick., Turn...","[[{'instruction': 'Drive east.', 'name': '-', ..."
3,3,"[[-81.4371032715, 28.2898006439], [-80.3951633...","Kissimmee Gateway Airport (ORLANDO, FL) to IND...","[[Drive east on Patrick Street., Turn left ont...",[[{'instruction': 'Drive east on Patrick Stree...
4,4,"[[-122.2488937, 38.11790375], [-122.8130035, 3...",ST. HELENA HOSPITAL CENTER FOR BEHAVIORAL HEAL...,"[[Drive north on Broadway Street., Turn left o...",[[{'instruction': 'Drive north on Broadway Str...
...,...,...,...,...,...
495,495,"[[-90.709503, 42.402], [-92.40029907226562, 42...","Dubuque Regional Airport (DUBUQUE, IA) to Wate...","[[Drive north., Bear right after 1129 meters o...","[[{'instruction': 'Drive north.', 'name': '-',..."
496,496,"[[-76.56655563780009, 42.9295192], [-76.106300...","Seward, William H., House (Auburn, NY) to Syra...","[[Drive north., Turn left onto South Street/NY...","[[{'instruction': 'Drive north.', 'name': '-',..."
497,497,"[[-71.06823934295548, 42.35671345], [-72.2428,...","Headquarters House (Boston, MA) to BAYSTATE MA...","[[Drive west on Beacon Street., Bear left towa...",[[{'instruction': 'Drive west on Beacon Street...
498,498,"[[-86.58465712669839, 34.73047047522626], [-87...","Milligan Block (Huntsville, AL) to Northwest A...","[[Drive northwest on East Side Square., Take t...",[[{'instruction': 'Drive northwest on East Sid...


#### Choose steps and add questoins and ground truth

In [2]:
df['Plan'] = df['Plan'].apply(lambda p: p[0])

random.seed(42)

def get_question_and_ground_truth(row):

    plan = row['Plan']
    actions = ['Turn', 'Keep']

    # Choosing step
    steps_with_actions = [i for i, step in enumerate(row['Plan']) if any(action in step for action in actions)]

    if steps_with_actions:
        step_idx = random.choice(steps_with_actions)
    else:
       return None
    
    step = plan[step_idx]

    # Flipping directions
    if 'left' in step:
        direction = 'right'
    elif 'right' in step:
        direction = 'left'
    else:
        return None

    if 'Keep' in step:
        action = 'keeping'
    else:
        action = 'turning'

    return (f"Does {action} {direction} in Step {step_idx + 1} taint the preconditions for the rest of the directions?", "Yes")


df[['Question', 'Ground Truth']] = df.apply(get_question_and_ground_truth, axis=1, result_type="expand")

df = df.dropna()

df = df.sample(n=100, random_state=42)

df


,id,Domain,Goal,Plan,Detailed Plan,Question,Ground Truth
361,361,"[[-71.1171723799392, 42.3860294], [-70.9115623...","Frost, Walter, House (Cambridge, MA) to Newman...","[Drive north on Frost Street., Turn left onto ...",[[{'instruction': 'Drive north on Frost Street...,Does turning left in Step 17 taint the precond...,Yes
73,73,"[[-117.1657314, 32.75477023], [-118.189273, 33...",UC SAN DIEGO HEALTH HILLCREST - HILLCREST MEDI...,"[Drive east on Dickinson Street., Turn right o...",[[{'instruction': 'Drive east on Dickinson Str...,Does keeping left in Step 8 taint the precondi...,Yes
374,374,"[[-85.78553679398806, 38.24351795], [-84.49642...","Hook and Ladder Company No. 5 (Louisville, KY)...","[Drive north., Turn left onto Garland Avenue.,...","[[{'instruction': 'Drive north.', 'name': '-',...",Does turning right in Step 13 taint the precon...,Yes
155,155,"[[-74.060799, 40.850101], [-74.45002013284314,...","Teterboro Airport (TETERBORO, NJ) to New Jerse...","[Drive east., Turn left onto Fred Wehran Drive...","[[{'instruction': 'Drive east.', 'name': '-', ...",Does turning right in Step 2 taint the precond...,Yes
104,104,"[[-87.02189636, 30.7241993], [-87.318604, 30.3...",Whiting Field Naval Air Station - North (MILTO...,"[Drive northeast on Whiting Field NAS North., ...",[[{'instruction': 'Drive northeast on Whiting ...,Does turning left in Step 9 taint the precondi...,Yes
...,...,...,...,...,...,...,...
347,347,"[[-112.03770104043008, 43.49026155], [-112.449...","Trinity Methodist Church (Idaho Falls, ID) to ...","[Drive southwest., Turn right onto Elm Street....","[[{'instruction': 'Drive southwest.', 'name': ...",Does turning left in Step 2 taint the precondi...,Yes
86,86,"[[-70.83377256632653, 42.6763836], [-70.932585...","Calef, Dr. John, House (Ipswich, MA) to Carney...","[Drive west on Poplar Street., Bear left onto ...",[[{'instruction': 'Drive west on Poplar Street...,Does turning right in Step 13 taint the precon...,Yes
75,75,"[[-96.86960835, 32.82592017], [-96.76907374, 3...","OUR CHILDRENS HOUSE (DALLAS, TX) to PLANO SPEC...","[Drive south., Turn left onto Brookriver Drive...","[[{'instruction': 'Drive south.', 'name': '-',...",Does turning right in Step 9 taint the precond...,Yes
438,438,"[[-71.17169802, 42.2746273], [-71.131674659716...",VA MEDICAL CENTER - WEST ROXBURY DIVISION (WES...,"[Drive south., Turn left after 117 meters or 1...","[[{'instruction': 'Drive south.', 'name': '-',...",Does keeping right in Step 8 taint the precond...,Yes


#### Sort and save data

In [3]:
df = df[[
    'id',
    'Goal',
    'Plan',
    'Question',
    'Ground Truth',
]]

df = df.sort_values(by=['id'])

df.to_csv('../output_data/data-flow_analyses/taint_analysis.csv', index=False)

df

,id,Goal,Plan,Question,Ground Truth
0,0,WESTWOOD/PEMBROKE HEALTH SYSTEM WESTWOOD (WEST...,"[Drive north on Abbey Road., Turn left onto Cl...",Does turning right in Step 2 taint the precond...,Yes
2,2,"CLARITY CHILD GUIDANCE CENTER (SAN ANTONIO, TX...","[Drive east., Turn left onto Tom Slick., Turn ...",Does turning right in Step 29 taint the precon...,Yes
9,9,"MBS International Airport (FREELAND, MI) to Be...","[Drive north on Flight Service Drive., Turn ri...",Does turning left in Step 15 taint the precond...,Yes
11,11,"Tuscaloosa National Airport (TUSCALOOSA, AL) t...","[Drive northeast., Turn left after 136 meters ...",Does turning right in Step 2 taint the precond...,Yes
15,15,"WHITTIER PAVILION (HAVERHILL, MA) to New Bedfo...","[Drive north., Turn left after 44 meters or 7-...",Does turning right in Step 2 taint the precond...,Yes
...,...,...,...,...,...
490,490,"Merrick-Simmons House (Fernandina Beach, FL) t...","[Drive north on South 10th Street., Turn left ...",Does turning left in Step 14 taint the precond...,Yes
491,491,"MILFORD HOSPITAL (MILFORD, CT) to Groton New L...","[Drive southeast., Turn left after 36 meters o...",Does turning right in Step 6 taint the precond...,Yes
494,494,NELL J. REDFIELD MEMORIAL HOSPITAL (MALAD CITY...,"[Drive south., Turn left onto Bannock Street.,...",Does turning right in Step 2 taint the precond...,Yes
495,495,"Dubuque Regional Airport (DUBUQUE, IA) to Wate...","[Drive north., Bear right after 1129 meters or...",Does keeping left in Step 15 taint the precond...,Yes
